# Prediction Algorithms

The main idea is to predict the next day low, upper, open and close to determine a strategy for the next day

In [1]:
import pandas as pd
from datetime import datetime
from datetime import date 
import numpy as np

from time import sleep
from tqdm.notebook import tqdm, trange

import seaborn as sns
import matplotlib as plt
import plotly.graph_objects as go

from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split as tts
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor as GBR

# Functions

In [2]:
#Conversion from strings to numeric
def strnum(x):
    try:
        x=x.replace(',','')
        return float(x)
    except:
        return x

#-----------movingAvg function-----------------
#Calculates the Moving Average

#df - list with the dataframes
#x - index of the coin, e.g. 0- BTC, 1-ETH... of the databases' list who contains the collections of data
#dname - name of the destination column of the moving average calculation (not creared yet)
#oname - name of the origin column of the data to take the information for the moving avarege
#msize - number of registers to calculate the moving average

def movingAvg(df,x,dname,oname,msize):
    df[x][dname] = 0.0
    for i in range(len(df[x])):
        temp=0.0
        if i<msize:
            pass
        else:
            for j in range(msize):
                temp+= data[x][oname][i-msize+j]
            df[x][dname][i]=temp/msize
#-----------------oheDate--------------------
#Makes one hot econfing to date depending day of the week, day of the month, week of year and month of year

def oheDate(df,x):
    df[x]['weekday']=df[x]['date']
    df[x]['weekday']=df[x]['weekday'].apply(lambda z: str(z.weekday()))
    
    #df[x]['calendarweek']=df[x]['date']
    #df[x]['calendarweek']=df[x]['date'].apply(lambda z: str(z.isocalendar()[1]))
    
    #df[x]['dayofmonth']=df[x]['date']
    #df[x]['dayofmonth']=df[x]['date'].apply(lambda z: str(z.day))
    
    #df[x]['month']=df[x]['date']
    #df[x]['month']=df[x]['date'].apply(lambda z: str(z.month))
    
    #Contanating dummies
    ohe=pd.get_dummies(df[x][['weekday']])#,'calendarweek']])#,'dayofmonth','month']])
    df[x]=pd.concat([df[x],ohe],axis=1)
    #Eliminating categorical columns
    df[x].drop(columns=['weekday'], inplace=True)
    
    
#--------------------yesterday-----------------
#Takes n "days" to the current day as new column (parameter) for the prediction
def yesterday(df,x,days=1,closev=True,openv=True,highv=True,lowv=True,volumev=True):
    
    #Values selector from the function inputs
    values_names=['close','open','high','low','volume']
    values=[closev,openv,highv,lowv,volumev]
    vals=[]
    for i,e in enumerate(values):
        if e==True:
            vals.append(values_names[i])
    yestvals=[]#List of added variables
    
    #Main set    
    for day in range(days):
        for val in vals:
            y=val+'_'+str(day)
            df[x]['yesterday_'+y]=0.0
            yestvals.append('yesterday_'+y)
            for i in range(len(df[x])-1):
                df[x]['yesterday_'+y][i+1]=df[x][val][i]
    
    return yestvals

# Load and Transform

In [3]:
#Global Variables
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']

#Opening All databases
data=[]
for i,e in enumerate(coins):
    file=e+'-mxn.csv'
    data.append(pd.read_csv(file))
#Conversion of dates from strin to timestamps
for i in range(len(coins)):
    data[i].date=data[i].date.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

# Conversion to numeric 
for i in range(len(coins)):
    data[i].open=data[i].open.apply(lambda x: strnum(x))
    data[i].close=data[i].close.apply(lambda x: strnum(x))
    data[i].low=data[i].low.apply(lambda x: strnum(x))
    data[i].high=data[i].high.apply(lambda x: strnum(x))
    data[i].volume=data[i].volume.apply(lambda x: strnum(x))

#MACD Calculation
metricas=['open','close','high','low','volume']

for i in tqdm(range(len(coins))):
    for m in metricas:#tqdm(metricas):
        movingAvg(data,i,'PMEF'+ m,m,12)
        movingAvg(data,i,'PMES'+m,m,26)
        data[i]['MACD'+m]=data[i]['PMEF'+m]-data[i]['PMES'+m]
        movingAvg(data,i,'MACD9'+m,'MACD'+m,9)
        data[i]['diffMACD'+m]=data[i]['MACD'+m]-data[i]['MACD9'+m]
    oheDate(data,i)
    yv=yesterday(data,i,14)



# Predictior Function

In [4]:
def predictor(data=data,end=0):
    #Create Dataframe for predictions
    preds= pd.DataFrame(columns=metricas)
    preds['coin'] =""
    cols=preds.columns.tolist()
    cols=cols[-1:]+cols[:-1]
    preds=preds[cols]
    preds

    for x in tqdm(range(len(coins))):
        test=data[x][35:len(data[x])-1-end] #35 is 26 + 9 for all the number of samples who doesn't has Moving Avg  
        dc=['date','open','close','low','high','volume']
        timevars = [ 'weekday_'+ str(x) for x in range(7)]

        #Adding new row for predictions
        preds=preds.append([0], ignore_index=True).drop(0,axis=1)
        preds=preds[cols]
        preds['coin'][x]=coins[x]

        for m in metricas:
            #Predictions
            X_train, X_test, y_train, y_test=tts(test[['PMEF'+m,'PMES'+m,'MACD'+m,'MACD9'+m]+timevars+yv],test[m], shuffle=True,random_state=100,test_size=.1)
            modelo=XGBR()
            modelo.fit(X_train, y_train)
            preds[m][x]=modelo.predict(data[x][len(data[x])-1-end:len(data[x])-end][['PMEF'+m,'PMES'+m,'MACD'+m,'MACD9'+m]+timevars+yv])[0]
            #train_score=modelo.score(X_train, y_train)
            #test_score=modelo.score(X_test, y_test)
            #print ('Score para', coins[x],m,train_score, test_score)

    #Getting oppenings to make initial corrections

    preds['ropen']=0.0
    for i,e in enumerate(coins):
        preds.ropen[i]=data[i]['open'][len(data[i])-1-end]

    preds['corr']=preds.ropen - preds.open
    preds['pcorr']= (preds.open/preds.ropen - 1)*100
    preds['pcorr']= preds['pcorr'].apply(lambda x: str(x.round(1))+'%')


    #Calculation of investment factors

    preds['go']= preds.close - preds.open
    preds['go']= preds['go'].apply(lambda x: 1 if x>0 else 0)
    preds['buy']=preds.open - (preds.open-preds.low)/3 # 1/3 below the opening regarding the lowest point
    preds['sell']=(preds.high-preds.close)/3+preds.close # 1/3 above the closure regarding the highes point
    preds['rate']=(preds.sell/preds.buy-1)*100
    preds.go=preds.go*preds.rate
    preds.go=preds['go'].apply(lambda x: 1 if x>0 else 0)
    preds.rate=preds.rate.apply(lambda x: x.round(1))

    #Calculuation correction

    preds['buya'] = preds['buy'] + preds['corr']
    preds['sella'] = preds['sell'] # + preds['corr']
    preds['rrate']= (preds['sella']/preds['buya'] - 1)*100
    preds['stoploss'] = preds['low'] + preds['corr']
    preds.rrate=preds.rrate.apply(lambda x: x.round(1))
    preds['go']=preds['rrate']
    preds['go']= preds['go'].apply(lambda x: 1 if x>2 else 0)
    
    return preds


In [5]:
preds=predictor()

# Display Predictions

In [6]:
sortedcols=['coin','buya','sella','stoploss','go','rrate']
preds[sortedcols]

,coin,buya,sella,stoploss,go,rrate
0,btc,207920,213841,207741,1,2.8
1,eth,5330.22,5457.18,5114.67,1,2.4
2,xrp,4.44316,4.55894,4.36949,1,2.6
3,ltc,997.776,993.036,989.788,0,-0.5
4,bch,5204.53,5292.05,4991.9,0,1.7
5,tusd,22.3407,22.5597,22.122,0,1.0
6,mana,0.92826,0.927613,0.924781,0,-0.1
7,gnt,1.24045,1.22593,1.20136,0,-1.2
8,bat,5.82977,6.19366,5.6293,1,6.2
9,dai,22.8403,22.7807,22.821,0,-0.3


In [7]:
preds

,coin,open,close,high,low,volume,ropen,corr,pcorr,go,buy,sell,rate,buya,sella,rrate,stoploss
0,btc,208060,213622,214277,207792,146.169,208010.01,-50.4744,0.0%,1,207971,213841,2.8,207920,213841,2.8,207741
1,eth,5444.4,5430.57,5510.4,5121.07,878.634,5438.00,-6.39746,0.1%,1,5336.62,5457.18,2.3,5330.22,5457.18,2.4,5114.67
2,xrp,4.49029,4.53983,4.59716,4.37979,8.10558e+06,4.48,-0.0102916,0.2%,1,4.45346,4.55894,2.4,4.44316,4.55894,2.6,4.36949
3,ltc,1005.62,982.555,1014,993.641,1262.75,1001.77,-3.8528,0.4%,0,1001.63,993.036,-0.9,997.776,993.036,-0.5,989.788
4,bch,5316.74,5284.79,5306.57,4997.8,165.625,5310.84,-5.90365,0.1%,0,5210.43,5292.05,1.6,5204.53,5292.05,1.7,4991.9
5,tusd,22.545,22.6653,22.3486,22.217,16343.7,22.45,-0.0950497,0.4%,0,22.4357,22.5597,0.6,22.3407,22.5597,1.0,22.122
6,mana,0.919402,0.923988,0.934865,0.914183,662409,0.93,0.0105982,-1.1%,0,0.917662,0.927613,1.1,0.92826,0.927613,-0.1,0.924781
7,gnt,1.26584,1.22252,1.23277,1.20719,206593,1.26,-0.00583767,0.5%,0,1.24629,1.22593,-1.6,1.24045,1.22593,-1.2,1.20136
8,bat,5.99917,6.1076,6.36577,5.69847,116584,5.93,-0.0691732,1.2%,1,5.89894,6.19366,5.0,5.82977,6.19366,6.2,5.6293
9,dai,22.7953,22.7402,22.8618,22.7664,8601.91,22.85,0.0546604,-0.2%,0,22.7857,22.7807,-0.0,22.8403,22.7807,-0.3,22.821


# Testing to find parameter to buy and sell

In [8]:
def check(n=1):
    
    metricas=['open','close','high','low','volume']
    diffs=[]
    
    for i in tqdm(range(n)):
        preds=predictor(data,i) #Getting the preductions of day i
        
        for c in range(8):
            for m in metricas:
                real=data[c][len(data[c])-i-1:len(data[c])-i].iloc[0][m]
                pred=(preds.iloc[c][m]).round(2)
                error=1-pred/real
                diffs.append([c,m,error.round(2),real,pred])
    return pd.DataFrame(diffs)

In [9]:
z=check(62)

All but volume has less than 2% of error in the prediction. Error in volume is huge

In [12]:
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']
metricas=['open','close','high','low','volume']
for i,e in enumerate(coins):
    for m in metricas:
        print(e,'in',m ,z[(z[0]==i) & (z[1]==m)][2].min().round(2)*100)

btc in open -2.0
btc in close -6.0
btc in high -4.0
btc in low -5.0
btc in volume -866.0
eth in open -2.0
eth in close -9.0
eth in high -4.0
eth in low -7.000000000000001
eth in volume -1023.0
xrp in open -2.0
xrp in close -8.0
xrp in high -6.0
xrp in low -7.000000000000001
xrp in volume -756.0
ltc in open -2.0
ltc in close -10.0
ltc in high -6.0
ltc in low -11.0
ltc in volume -236.0
bch in open -2.0
bch in close -10.0
bch in high -9.0
bch in low -13.0
bch in volume -4753.0
tusd in open -3.0
tusd in close -3.0
tusd in high -4.0
tusd in low -2.0
tusd in volume -365.0
mana in open -4.0
mana in close -10.0
mana in high -11.0
mana in low -6.0
mana in volume -933.0
gnt in open -3.0
gnt in close -9.0
gnt in high -28.000000000000004
gnt in low -8.0
gnt in volume -960.0


AttributeError: 'float' object has no attribute 'round'

In [ ]:
z[(z[0]==7) & (z[1]=='open')]

In [ ]:
z

# Algorthms Intraday Trading

# Graphic

In [ ]:
#Graphic with Candle Sticks
i=0
fig=go.Figure(data=[go.Candlestick(x=data[i].date,
                                   open=data[i].open,
                                   high=data[i].high,
                                   low=data[i].low,
                                   close=data[i].close
                                  )
                   ]
             )
fig.show()